    imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import re
import xgboost as xgb
from sklearn.model_selection import train_test_split
import sklearn
import random
import catboost
from catboost import CatBoostRegressor, Pool, CatBoostClassifier
%matplotlib inline

    Downloading Data. 
    
    All files and folders are extracted from zip and placed in the same folder as .ipynb 
    
    Creating df which includes text data. No NaNs
    
    regulations_texts.csv is used for df['text'], 'ria_reports/ria_reports/ria_reports_main.csv'  for  df['co_developer']
    
    After that, only df is used

In [2]:
df = pd.read_csv("regulations.csv")
df_texts = pd.read_csv("regulations_texts.csv", sep=";")
answers = pd.read_csv("train_answer.csv")
df_reports = pd.read_csv('ria_reports/ria_reports/ria_reports_main.csv', sep=";")

df['publication_date'] = pd.to_datetime(df['publication_date'])
df['text'] = [df_texts[df_texts['regulation_project_id'] == x].iat[0,1] 
              if x in set(df_texts['regulation_project_id']) else 'nothing' for x in df['id']]
df['co_developer'] = [df_reports[df_reports['regulation_project_id'] == x].iat[0,3] 
              if x in set(df_reports['regulation_project_id']) else 'nothing' for x in df['id']]

#Get rid of nans
for col in df:
    #get dtype for column
    dt = df[col].dtype 
    #check if it is a number
    if dt == int or dt == float or dt == 'float64':
        df[col] = df[col].fillna(0)
    else:
        df[col] = df[col].fillna("nothing")

len(df_texts)

11384


    Preparing Data
    
    Creating df1_cat_dev which has only numeric data

    Actually the main function data_proc(df, answers) was used to return 3 tables: df1, df1_cat, df1_cat_dev. But only the last one was used for the final submissions.
    df1 - fully numerical
    df1_cat - with some categorical features
    df1_cat_dev - with lots of categorical features (including developers)
    
    df1 and df1_cat were deleted from function for simplicity
    That's the reason why df1_cat_dev has such a strange name!
    
    df1_cat_dev has 24 features and an 'id' column which will be deleted later.
    
added_by

responsible

year

developer

regulatory_impact

mineco_solution

views_num

month

npa_type

act_objectives

year_end

text_len

co_dev_mean

dislikes_num

проект

act_significance

persons_affected_by_act

text

problem_addressed

is_regionally_signigicant

okved_list

relations_regulated_by_act

co_developer_bool

act_changes_controlling_activities

Features are ordered according to catboost feature_importances_

In [19]:
def co_dev_columns(X, y):
    '''
        Dealing with "co_developers"
        Creating features:
            'co_developer_count' - the number of  co_developers
            'co_dev_mean' - mean success for co_developers (mean from the means)
            'co_developer_bool' - are there any co_devs
    '''
    
    X = X.copy()
    X['answer'] = y
    
    X['co_developer_bool'] = [0 if x.lstrip().lower() in ['nothing', 'нет', 'отсутствуют', '-', 'соисполнителейнет', 
                                                         'соисполнителей-нет','соисполнителиотсутствуют'] else 
                            1 for x in df['co_developer']]
    
    X['co_dev_mean'] = [0 for i in range(len(X))]
    X['co_developer_count'] = [0 for i in range(len(X))]
    for x in df['developer'].unique():
        if x != 'nothing':
            X['co_dev_' + x] = [1 if x in t else 0 for t in df['co_developer']]
            X['co_developer_count'] = X['co_developer_count'] + X['co_dev_' + x]
            if X['co_dev_' + x].sum() != 0:
                succ_rate = (X['co_dev_' + x]*X['answer']).sum()/X['co_dev_' + x].sum()
                X['co_dev_' + x] = [succ_rate if t != 0 else 0 for t in X['co_dev_' + x]]
                X['co_dev_mean'] = X['co_dev_mean'] + X['co_dev_' + x]

            del X['co_dev_' + x]

    X['co_dev_mean'] = X['co_dev_mean']/X['co_developer_count']
    X['co_dev_mean'] = [x if x < 10000 else 0 for x in X['co_dev_mean']]
    del X['co_developer_count']
    del X['answer']
    return X


def data_proc(df, answers):
    '''
        Creating the table df1_cat_dev for catboost
    '''
    df1 = df.loc[:,['id', 'publication_date', 'act_title', 'developer', 'okved_list', 'views_num', 
                    'dislikes_num', 'regulatory_impact', 
                    'is_regionally_signigicant', 'act_changes_controlling_activities', 'mineco_solution',
                    'problem_addressed', 'act_objectives', 'persons_affected_by_act',
                    'relations_regulated_by_act','act_significance']]

    id_dict = dict(zip(answers['id'],answers['passed']))
    y = np.array([id_dict.get(i, np.nan) for i in df1['id']])
    
    df1['text'] = [0 if x == 'nothing' else 1 for x in df['text']]   #Does the text for this is exist
    df1['text_len'] = [0 if x == 'nothing' else len(x) for x in df['text']]    #Length of the text, 0 if no text
    df1.loc[:,['is_regionally_signigicant']] = [1 if i == True else 0 for i in df1['is_regionally_signigicant']]
    df1.loc[:,['act_changes_controlling_activities']] = [1 if i == True else 0 
                                                         for i in df1['act_changes_controlling_activities']]

    #bool features from corresponding columns (empty or not)
    df1.loc[:,['problem_addressed', 'act_objectives', 'persons_affected_by_act', 'relations_regulated_by_act',
               'act_significance']] = df1.loc[:,['problem_addressed', 'act_objectives', 'persons_affected_by_act',
                    'relations_regulated_by_act','act_significance']].applymap(lambda x: 0 if x=='nothing' else 1)
    df1['okved_list'] = [0 if x=='nothing' else 1 for x in df1['okved_list']]
    df1['year'] = [0 if x=='nothing' else x.year for x in df['publication_date']]    # Number of the year
    df1['month'] = [0 if x=='nothing' else x.month for x in df['publication_date']]   # Number of the month
    
    #months to yeAR_end. 12 max     Linear decline as time limit becomes closer.
    df1['year_end'] = [ min(2021*12 + 9 - df1['year'][i]*12 - df1['month'][i], 12) for i in df1.index]

    #Dealing with persons (added_by and responsible)
    df1['same_person'] = (df['added_by'] == df['responsible'])
    responsible = pd.DataFrame(np.unique(df['responsible'], return_counts=True)).transpose()
    responsible = dict(zip(responsible[0], responsible[1]))
    responsible['nothing'] = 0
    added_by = pd.DataFrame(np.unique(df['added_by'], return_counts=True)).transpose()
    added_by = dict(zip(added_by[0], added_by[1]))
    added_by['nothing'] = 0
    df1['responsible_count'] = [responsible[x] for x in df['responsible']]
    df1['added_by_count'] = [added_by[x] for x in df['added_by']]
    df1['co_developer_bool'] = [0 if x.lstrip().lower() in ['nothing', 'нет', 'отсутствуют', '-', 'соисполнителейнет', 
                                                         'соисполнителей-нет','соисполнителиотсутствуют'] else 
                            1 for x in df['co_developer']]
    
    
    
    def npa_type(s):
        '''
            Extracting the type of NPA from the text
        '''
        lines = s.replace('|', '').split('\n')
        todrop =[]
        for i in range(len(lines)):
            if lines[i].lstrip()=='':
                todrop.append(i)
        lines = np.array(lines)
        lines = np.delete(lines, todrop)
        lines = [re.sub(' +', ' ', x).strip(string.punctuation).lstrip().lower().replace(' ', '') for x in lines]
        for i in ['приказ', 'федеральныйзакон', 'постановление', 'указ']:
            if i in set(lines[:min(len(lines), 10)]):
                return i
        return 'nothing'

    df1['npa_type'] = [npa_type(x) for x in df['text']]


    
    

    def first_word(table, word):
        '''
            adding feature that is the 1st word in act_title (1 if it's equal to word, 0 - else)
            This function was created before npa_type(s) and was used for top 10 frequently used words. But 'проект'
            proved its' significance even after npa_type appearance.
        '''
        table[word] = [1 if x.split()[0].strip(string.punctuation).strip('«').lower() == word else 0 
                       for x in table['act_title']]  

    # top_first_words = 10

    # A = pd.DataFrame(df['act_title'])
    # A[0] = [x.split()[0].strip(string.punctuation).strip('«').lower() for x in A['act_title']]
    # for word in pd.DataFrame(np.unique(A[0], return_counts=True)
    #                          ).transpose().sort_values(1, ascending=False)[0][:top_first_words].values:
    #     first_word(df1, word)
    first_word(df1, 'проект')
    
    
    
    
    del df1['responsible_count']
    del df1['added_by_count']
    del df1['same_person']
    
    del df1['act_title']
    del df1['publication_date']
    
    df1 = co_dev_columns(df1, y)
    df1_cat_dev = df1.copy()
    df1_cat_dev['added_by'] = df['added_by']
    df1_cat_dev['responsible'] = df['responsible']

    
    return df1_cat_dev, id_dict, y
    




df1_cat_dev, id_dict, y = data_proc(df, answers)

len(df1_cat_dev.columns)


25

    learning
    
    Initially both xgBoost and catboost were used. But xgBoost was deleted from final version due to the worse results.

In [20]:
def learn(df1, answers, method='cat', seed=111111, iterations=1500, learning_rate=0.1,
          depth=None, cat_num_features=['year', "year_end", 'month']):
    '''
        Constructing the model
        Fitting the answers
        Returning AUC and F1 scores
    '''

    
    X = df1[df1['id'].isin(answers['id'])]
    
    id_dict = dict(zip(answers['id'],answers['passed']))
    y = np.array([id_dict[i] for i in X['id']])
    
    #Only npa with text were used for testing
    random.seed(seed)
    X_text = X[X['text']==1]
    X_test = X.loc[random.sample(list(X_text.index), len(X_text)//2), :]
    
    
    y_test = np.array([id_dict[i] for i in X_test['id']])
    X_train = X[~X['id'].isin(X_test['id'])].copy()
    y_train = np.array([id_dict[i] for i in X_train['id']])
    
    
    print(f'Len of X_train is: {len(X_train)}')
    print(f'Len of X_test is: {len(X_test)}')
    
    del X_test['id']
    del X_train['id']  
    del X['id']
    
    categorical_features = [x for x in X_train.columns if X_train[x].dtype == 'object'] + cat_num_features
    
    if method=='cat':

        xg_cl = CatBoostClassifier(iterations=iterations,random_state=seed, eval_metric='AUC',
                                   cat_features=categorical_features, verbose=False, 
                                   learning_rate=learning_rate, depth=depth)
        #xg_cl.fit(X_train, y_train)   #Used for testing
        xg_cl.fit(X, y)   #Used for submit



    #Printig the results on train
    predict = xg_cl.predict(X_train)
    predict_proba = xg_cl.predict_proba(X_train)

    print(f'Train F1: {sklearn.metrics.f1_score(y_train, predict)}')
    print(f'Train roc_auc_score: {sklearn.metrics.roc_auc_score(y_train, pd.DataFrame(predict_proba)[1])}')

    #Printig the results on test
    predict = xg_cl.predict(X_test)
    predict_proba = xg_cl.predict_proba(X_test)
    auc = sklearn.metrics.roc_auc_score(y_test, pd.DataFrame(predict_proba)[1])

    print(f'Test F1: {sklearn.metrics.f1_score(y_test, predict)}')
    print(f'Test roc_auc_score: {sklearn.metrics.roc_auc_score(y_test, pd.DataFrame(predict_proba)[1])}')
    
    #Submitting results to result.csv
    X_res = df1[~df1['id'].isin(answers['id'])]
    Result = pd.DataFrame(X_res['id'].copy()).reset_index(drop=True)
    del X_res['id']
    predict_proba = xg_cl.predict_proba(X_res)
    Result['passed'] = pd.DataFrame(predict_proba)[1]
    Result.to_csv("result.csv", index=False)
    
    return xg_cl, X, y, X_res, X_test, y_test, categorical_features, auc

xg_cl, X, y, X_res, X_test, y_test, categorical_features, auc = learn(df1_cat_dev, answers, method='cat', seed=6005,
                                                                     iterations=900, learning_rate=0.09, depth=None,
                                                                     cat_num_features=['year'])

Len of X_train is: 77314
Len of X_test is: 3692
Train F1: 0.7636327971753628
Train roc_auc_score: 0.9716835528539544
Test F1: 0.8416666666666667
Test roc_auc_score: 0.9897952497441402
